In [1]:
import os
from keras.models import *
from keras.optimizers import *
from keras.layers import *
import numpy as np
import sys
from DataProcessor import *

#the main and only function of the RNN_Model

def ApplyRNNModel(X,Y):

    #hidden layers and batch sizes
    HIDDEN_SIZE1 = 512
    HIDDEN_SIZE2 = 256
    BATCH_SIZE = 64


    print('Build model...')

    #use LSTM layers
    RNN = LSTM
    model = Sequential()

    #First RNN layer with input (number of collectors * len of the sequence)
    model.add(RNN(HIDDEN_SIZE1, input_shape=(len(X[0]), len(X[0][0]))))
    model.add(Reshape(target_shape=(16,32)))
    #Second RNN layer with the same input again
    model.add(RNN(HIDDEN_SIZE2, input_shape=(len(X[0]), len(X[0][0]))))
    model.add(Dropout(0.4))
    #last Fully connected layer
    model.add(Dense(HIDDEN_SIZE2))
    model.add((Dense(len(Y[0]))))
    #Softmax to select based on probabilities
    model.add(Activation('softmax'))

    #optimize using gradient descent
    sgd = SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    #to view a summary that represents the full RNN model
    model.summary()


    #next line: to be called if a model is saved with the same attributes
    model.load_weights('model')

    #run model on data (1000 times) with random 1% to validate the model
    model.fit(X, Y, batch_size=BATCH_SIZE, nb_epoch=1000, validation_split=0.1)

    #built in keras function to save weights
    model.save_weights('model')

    


#the RNN_Model start running here
processor=DataProcessor() #create object
processor.Generate_X_Y_Vectors(path='Data.csv') #call generation function from the data processor file
X,Y=processor.X,processor.Y #get X,Y matrices from the processor object
ApplyRNNModel(X,Y) #call the RNN function
